In [ ]:
from fastrag.utilities.qdrant_database import QdrantDatabase

In [ ]:
from qdrant_client import QdrantClient, models

qdrant_database = QdrantDatabase(qdrant_url="http://0.0.0.0:6643")


In [ ]:
collection_name = "qna_collection"
# qdrant_database.delete_collection(collection_name)
qdrant_database.create_collection(collection_name)

In [ ]:
qdrant_database.load_recent_responses(collection_name=collection_name, limit=10)

In [ ]:
# qdrant_database.add_response(collection_name, "what are you doing?", "I am studying")
qdrant_database.add_response(collection_name, "tell me about Nepal", "It is a beautiful country.")

In [ ]:
import os
from qdrant_client import QdrantClient, models

qdrant_url = os.getenv("QDRANT_URL", "http://0.0.0.0:6333")

qdrant_client = QdrantClient(url=qdrant_url)



In [ ]:
import time

payload = {
    "user": "who are you?",
    "assistant": "I am somebody",
    "timestamp": int(time.time())
}

payload.__str__()


In [ ]:
import uuid

def create_uuid(text: str = None) -> str:
    import hashlib
    hash_bytes = hashlib.sha256(text.encode('utf-8')).digest()
    return str(uuid.UUID(bytes=hash_bytes[:16]))

create_uuid(payload.__str__())

In [ ]:
point_data = models.PointStruct(id=payload['timestamp'], vector=[0], payload=payload)
point_data

In [ ]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=[point_data]
)

In [ ]:
search_results[0]

In [ ]:
search_results = qdrant_client.scroll(collection_name=collection_name, with_payload=True)
search_results
# sorted_responses = sorted(
#     [result.payload for result in search_results[0]],
#     key = lambda x: x["timestamp"],
#     reverse=True
# )

# # return_value = [response["text"] for response in sorted_responses[:10]]
# return_value

In [ ]:
import requests
def add_conversation_history(query, response_text, collection_name:str="qna_collection"):
    try:
        url = os.getenv("CONVERSATION_HISTORY_URL", "http://localhost:8089/conversation-history")
        payload = {
            "collection_name": collection_name,
            "query": query,
            "response_text": response_text
        }
        response = requests.post(url, json=payload)
        print(response.json())
        
    except Exception as err:
        print(f"Failed to add to conversation history: {err}")
        return []    
    
add_conversation_history("are you working?", "Lets sleep now")

In [ ]:
def get_conversation_history(collection_name: str="qna_collection", limit: int=10):
    try:
        url = os.getenv("CONVERSATION_HISTORY_URL", "http://localhost:8089/conversation-history")
        payload = {
            "collection_name": collection_name,
            "limit": limit
        }
        response = requests.get(url, params=payload)
        conversation_history = response.json()["response"]

        chat_history = []
        for record in conversation_history:
            chat_history += [[record['user'], record['system']]]
        return chat_history
    except Exception as err:
        print(f"Error in accessing conversation history from database: {err}")
        return []
get_conversation_history()

In [ ]:
import requests
import os

url = os.getenv("CONVERSATION_HISTORY_URL", "http://localhost:8089/conversation-history")
payload = {
    "collection_name": "qna_collection",
    "limit": 10
}
response = requests.get(url, params=payload)
print(response.json())
conversation_history = response.json()["response"]